Lecture:


## Task 1

In [ ]:
from scipy import *
from pylab import *
import sys


def schurform(A):
'''
Compute the Schurform of a matrix A 

:param A: matrix (symmetric -> only real entries on the diagonal of the Schurmatrix)
:return: Schurform of A 
'''
i = 10000
    while i>0: #maybe this kind of loop is better than an for
        Q, R = np.qr(A)
        A = np.dot(R,Q)
        i -= 1
        
#####


def hessenberg(A):
    """
    Reduces a matrix A to Hessenberg form. If A is hermitian (symmetric), 
    the Hessenberg form will be tridiagonal.
    """
    m = shape(A)[0]
    for k in range(m-2):
        x = A[k+1:, k]
        v_k = sign(x[0])*array([norm(x, 2)]+(len(x)-1)*[0.]) + x      
        v_k = v_k/norm(v_k, 2)
        b = dot(v_k,A[k+1:,k:])
        A[k+1:,k:] -= 2*outer(v_k,b)
        c = dot(A[0:,k+1:],v_k.T)
        A[0:,k+1:] -= 2*outer(c,v_k)
    #A[abs(A) < 1.e-15] = 0 #will it disturb anything?
    return A
    
        

def schurform(A):
    '''
    Compute the Schurform of a matrix A 

    param A: matrix (symmetric -> only real entries on the 
    diagonal of the Schurmatrix)
    return: Schurform of A 
    '''
#suppose A symmetric
    A = hessenberg(A)
    A_0 = A
    n = shape(A_0)[0]
    eigvals = []
    maxit = 1000
    for i in range(maxit):
        mu_i = A_0[n-1,n-1] #why this choice
        q_i, r_i = qr(A_0-mu_i*eye(n))
        A_0 = r_i@q_i + mu_i*eye(n)
        for j in range(n-1):
            if abs(A_0[j,j+1]) < 1.e-8:
                A_0[j,j+1] = 0
                A_0[j+1,j] = 0
                A_1 = A[:j,:j]
                n_1 = shape(A_1)[0]
                A_2 = A[j+1:,j+1:]
                n_2 = shape(A_2)[0]
                for k in range(maxit): #trying to realise the deflation, but I its not right yet
                    mu_1 = A_1[n_1-1,n_1-1]
                    mu_2 = A_2[n_2-1,n_2-1]
                    q_1, r_1 = qr(A_1-mu_1*eye(n_1))
                    q_2, r_2 = qr(A_2-mu_2*eye(n_2))
                    A_1 = r_1@q_1 + mu_1*eye(n_1)
                    A_2 = r_2@q_2 + mu_2*eye(n_2)          
    return A



A = rand(5,5)
A_sym = (A + A.T)/2
print(A_sym)
print(hessenberg(A_sym))
print(schurform(A_sym))

## Task 2

To proof that (A-$\lambda$I) has rank at least m-1 if A$\in$ R$^{m x m}$ we can use that A is a tridiagonal and symmetric matrix with all its sub- and superdiagonal entries nonzero.
What we can use for the proof is that we know: a matrix has always rank greater or equal than it's submatrices. If we take the submatrix of our tridiagonal submatrix we will get a matrix with full rank (m-1), because if we take the submatrix S by deleting the first row and the last column: 
$ A = \begin{pmatrix}
d_{1} & c_{1} & 0 &... & 0 \\
a_{1} & d_{2} & c_{2} & \ddots &\vdots \\
    0 & a_{2}&    d_{3}   &   \ddots       &0 \\
\vdots & \ddots & \ddots& \ddots   & c_{m-1} \\
0 & \dots & 0& a_{m-1} & d_{m}
\end{pmatrix}$
it will look like this: 
$ S= \begin{pmatrix}
a_{1} & d_{2} & c_{2} & 0\\
     & a_{2}&    d_{3}   &  c_{m-2}\\
    & & \ddots & d_{m-1}\\
0 &  & & a_{m-1}   
\end{pmatrix}$
This is a upper triangular matrix where we already know that all diagonal entries are nonzero. Thus the matrix has full rank. 
But we also know that the (A-$\lambda$I) can't have full rank because this is the condition that $\lambda$ is an eigenvalue of A. With this we can say for sure A-$\lambda$I has the rank m-1 for each eigenvalue. If this applies for all eigenvalues they should all be distinct otherwise the matrix A-$\lambda$I wouldn't have rank m-1 for each eigenvalue.  

## Task 3

https://gist.github.com/swvist/3775568

In [12]:
import numpy as np
import scipy.linalg as sl
def characteristic_polynom(A, x):
    m = np.size(A)[0]
    if m > 0: 
        return (A[m-1;m,m-1;m]-x)*characteristic_polynom(A[:m-1,:m-1],x)-A[m-1:m,m-2:m-1]**2*characteristic_polynom(A[:m-2,:m-2],x)
    if m = 0:
        return 1


In [23]:
A = np.random.rand(5,5)
A = (np.transpose(A) + A)
A = sl.hessenberg(A)
print(A)
A[3:4,4:5]

[[  1.52154182e+00  -2.16926862e+00  -1.60624337e-16  -6.09617649e-17
    1.11004963e-16]
 [ -2.16926862e+00   3.40770622e+00   3.92615722e-01   6.92861385e-17
   -3.54251526e-16]
 [  0.00000000e+00   3.92615722e-01   6.35291030e-01  -3.68837296e-01
    7.35263474e-17]
 [  0.00000000e+00   0.00000000e+00  -3.68837296e-01   1.18261486e+00
    5.66827133e-01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   5.66827133e-01
   -6.78809385e-01]]


array([[ 0.56682713]])